In [1]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

import sys
np.set_printoptions(threshold=sys.maxsize)

np.random.seed(14)

In [2]:
def lin2db(x):
    return 10.0*np.log10(x)

def db2lin(x):
    return 10.0**(x/10.0)

def lin2dbm(x):
    return 10.0*np.log10(x)+30.0

def dbm2lin(x):
    return 10.0**(x/10.0 - 3.0)

In [3]:
def eCDF(data):
    x = np.sort(data)
    y = np.arange(0, len(data)) / len(data)

    return x, y 

In [4]:
total_bandwidth = 100e6 # [Hz]
tx_power = 1 # [W]

environment_constant = 1e-4
pathloss_constant = 4

ref_distance = 1 # [m]

In [5]:
def random_ue_positions(num_ue, cov_side):
    ''' 
    Returns a random position vector within the coverage area.
    
    Parameters
    ----------
    cov_side : int, float
        The side [in m] of the coverage area.
    num_ue : int
        The number of UEs.
    '''
    
    ue_positions = np.zeros((num_ue, 2))
    
    for ue in range(num_ue):
        ue_positions[ue] = [np.random.rand() * cov_side, np.random.rand() * cov_side]
        
    return ue_positions

In [6]:
def ap_positions(num_ap, cov_side):
    ''' 
    Returns the APs positions based on the number of APs.
    
    Parameters
    ----------
    num_ap : int
        The number of APs.
    cov_side : int, float
        The side [in m] of coverage area.
    '''
    
    if np.sqrt(num_ap).is_integer():
        
        side_ap_quantity = int(np.sqrt(num_ap))
        
        ap_area_side = cov_side // side_ap_quantity
        
        x_pos, y_pos = np.meshgrid(np.arange(0.5 * ap_area_side,
                                             cov_side,
                                             ap_area_side),
                                   np.arange(0.5 * ap_area_side,
                                             cov_side,
                                             ap_area_side))
 
        aps_pos = np.column_stack((x_pos.ravel() + y_pos.ravel()*1j)).reshape((num_ap, 1))
    
        return aps_pos
    
    else:
        
        print('Number of APs must be a perfect square')

In [7]:
def distance(ue_pos, ap_pos, ref_distance=1):
    ''' 
    Returns the distance [in m] between an UE and an AP. If distance is lower than reference distance, it returns the reference distance.
    
    Parameters
    ----------
    ue_pos : int, float
        The UE position.
    ap_pos : int, float
        The AP position.
    ref_distance : int, float
        The reference distance [in m] from which the gain is calculated.
    '''

    dis_matrix = np.zeros((ue_pos.shape[0], ap_pos.shape[0]))
    
    for ue in range(ue_pos.shape[0]):
        for ap in range(ap_pos.shape[0]):
            dis = np.absolute(ue_pos[ue] - ap_pos[ap]) 
            
            if dis[0] >= ref_distance:
                dis_matrix[ue, ap] = dis[0]
            
            else:
                dis_matrix[ue, ap] = ref_distance

    return dis_matrix

In [8]:
def shadowing(std, num_ue, num_ap):

    return np.random.lognormal(sigma = std, size = (num_ue, num_ap))

In [9]:
def rayleigh(std, num_ue, num_ap, num_ch):

    return np.sqrt((std*np.random.randn(num_ch, num_ue, num_ap))**2 + (std*np.random.randn(num_ch, num_ue, num_ap))**2) 

In [10]:
def channel(distance, shadowing, rayleigh):

    channel_mat = shadowing * (environment_constant / (distance ** pathloss_constant)) * rayleigh**2

    if rayleigh.shape[0] > 1:
        
        for row in range(rayleigh.shape[1]):
    
            channel_mat[np.random.randint(0, rayleigh.shape[0]), row, :] = np.zeros(rayleigh.shape[2])

    for row in range(rayleigh.shape[1]):

        max_channel = np.max(channel_mat[:, row, :])
        
        for col in range(rayleigh.shape[2]):

            if channel_mat[:, row, col] != max_channel:

                channel_mat[:, row, col] = 0

    return channel_mat

In [29]:
def sinr(power_vector, channel, noise_p):
    
    sinr_matrix = np.zeros(channel.shape)
    sinr_vector = np.zeros(channel.shape[1])

    
    for ch in range(channel.shape[0]):

        for col in range(channel.shape[2]):

            print(f'AP {col} \n')
        
            channel_sum = 0
            
            for row in range(channel.shape[1]):

                channel_sum += sum(np.abs(channel[ch, row, :]) * power_vector[row][0])
            
            for row in range(channel.shape[1]):
                
                interest_channel = power_vector[row][0] * np.abs(channel[ch, row, col])

                print(f'UE {row+1} \n interest channel: {interest_channel} \n interference: {channel_sum - interest_channel} \n')
                
                sinr_matrix[ch, row, col] = interest_channel / ((channel_sum - interest_channel) + noise_p)

    
    for row in range(channel.shape[1]):

        sinr_vector[row] = sinr_matrix[:, row, :].max()

    return lin2db(sinr_vector)

In [12]:
def capacity(total_bandwidth, num_ch, sinr):
    
    return ((total_bandwidth / num_ch) * np.log2(1 + db2lin(sinr))) / 1e6

In [13]:
def power_noise(total_bandwidth, num_ch):
    
    return (total_bandwidth/num_ch) * 1e-20

In [14]:
num_ue = 4
num_ap = 4
num_ch = 1

cov_side_noise = 1000 # [m]
cov_side_interference = 100 # [m]
seeds = 1

sh_std = 2
mp_std = 1 / np.sqrt(2) 

In [15]:
p_max = 30 # [dBm]
p_min = 0 # [dBm]

total_time = 100

step = 1 # [dBm]
sinr_target = 1 # [dB]

In [30]:
noise_p = power_noise(total_bandwidth, num_ch)

total_sinr_npc = np.zeros((num_ue, seeds))
total_sinr_ud = np.zeros((num_ue, seeds))
total_sinr_esnp = np.zeros((num_ue, seeds))

total_capacity_npc = np.zeros((num_ue, seeds))
total_capacity_ud = np.zeros((num_ue, seeds))
total_capacity_esnp = np.zeros((num_ue, seeds))

each_power_evolution_ud = np.zeros((num_ue, total_time, seeds))
total_power_evolution_ud = np.zeros((total_time, seeds))

each_sinr_evolution_ud = np.zeros((num_ue, total_time, seeds))

total_EE_npc = np.zeros(seeds)
total_EE_ud = np.zeros(seeds)

for seed in range(seeds):

    # Study case values
    
    ue_pos = np.array([[22.538 + 20.333j], 
                       [56.679 + 32.188j], 
                       [76.551 + 14.688j], 
                       [26.595 + 70.239j]])
    
    ap_pos = ap_positions(num_ap, cov_side_interference)
    distances = distance(ue_pos, ap_pos)
    
    shadowing_matrix = np.array([[5.3434e-2, 2.8731e-1, 1.9691e-2, 7.3013e-1], 
                                 [3.2318, 1.5770, 2.6449e-1, 5.6379], 
                                 [6.1470e-3, 1.1424, 2.6826e-1, 4.5709], 
                                 [1.3485e-1, 4.6690e-1, 7.8250e-1, 1.6742]])
     
    rayleigh_matrix = np.array([[1.248699, 3.248041, 0.772754, 0.708962], 
                                [0.498887, 0.104890, 0.647280, 0.940906], 
                                [0.382966, 0.682700, 1.891256, 0.327100], 
                                [0.065737, 0.649500, 1.981107, 1.259538]]).reshape((num_ch, num_ue, num_ap))

    
    # Channel coefficients matrix
    
    random_matrix = channel(distances, shadowing_matrix, rayleigh_matrix)

    
    # No power control (or max power)
    
    npc_power = p_max * np.ones(num_ue).reshape((1, random_matrix.shape[1])).T    
    
    for i in range(3):

        npc_power += 1
        
        # SINR 
        
        system_sinr_npc = sinr(dbm2lin(npc_power), random_matrix, noise_p)

AP 0 

UE 1 
 interest channel: 1.3530753029143341e-08 
 interference: 6.140983288254511e-07 

UE 2 
 interest channel: 0.0 
 interference: 6.276290818545945e-07 

UE 3 
 interest channel: 0.0 
 interference: 6.276290818545945e-07 

UE 4 
 interest channel: 0.0 
 interference: 6.276290818545945e-07 

AP 1 

UE 1 
 interest channel: 0.0 
 interference: 6.276290818545945e-07 

UE 2 
 interest channel: 0.0 
 interference: 6.276290818545945e-07 

UE 3 
 interest channel: 5.668600087769122e-09 
 interference: 6.219604817668253e-07 

UE 4 
 interest channel: 0.0 
 interference: 6.276290818545945e-07 

AP 2 

UE 1 
 interest channel: 0.0 
 interference: 6.276290818545945e-07 

UE 2 
 interest channel: 0.0 
 interference: 6.276290818545945e-07 

UE 3 
 interest channel: 0.0 
 interference: 6.276290818545945e-07 

UE 4 
 interest channel: 6.082961059822658e-07 
 interference: 1.933297587232868e-08 

AP 3 

UE 1 
 interest channel: 0.0 
 interference: 6.276290818545945e-07 

UE 2 
 interest chan